# README
well max cell filter for single date, **multisess**

In [34]:
import os
import scipy.io as sio
from tqdm import tqdm

import numpy as np
import scipy.stats as stats
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load metadata

In [8]:
dir_repo = r'C:\Users\ll357\Documents\inter/'.replace('\\', '/')
os.chdir(dir_repo)
from src import adp

dir_data = r'\\duhs-user-nc1.dhe.duke.edu\dusom_glickfeldlab\All_Staff\home\lan\Data\2P_images\mat_inter/'.replace('\\', '/')
dir_xls = os.path.join(dir_data, 'adp_dataset_master.xlsx')
data_info = pd.read_excel(dir_xls)

meta = data_info[(data_info.paradigm == 'grating_8ori_2isi_multisess') 
                 & (data_info.date == 240229)
                #  & ((data_info.cellpose_seg == True) | (data_info.manual_seg == True)) # ensure segmentation
                 ].reset_index(drop=True)
# nset = meta.shape[0]
nset = meta.date.nunique()
meta

,mouse,date,area,depth,num,cellpose_seg,manual_seg,paradigm,gcamp,time,AWS,note
0,1381,240229,V1,150.0,002,1.0,NaN,grating_8ori_2isi_multisess,6s,NaN,NaN,NaN
1,1381,240229,V1,150.0,003,1.0,NaN,grating_8ori_2isi_multisess,6s,NaN,NaN,NaN


# batch load df_tidy

In [40]:
df_tidy = pd.DataFrame()

for iset in tqdm(range(nset)):
    # print(f'iset={iset}, nset={nset}')

    mouse = meta.loc[iset, 'mouse'].astype(str)
    imouse = 'i' + mouse
    date = meta.loc[iset, 'date'].astype(str)
    area = meta.loc[iset, 'area']
    # sess = meta.loc[iset, 'num']
    # print(imouse, date, area, sess)
    dir_identifier = f'{area}_{imouse}_{date}_cellpose' # multisess. TODO: fix hard code
    # dir_identifier = f'{area}_{imouse}_{date}_{sess}' # single sess

    dir_data = r'\\duhs-user-nc1.dhe.duke.edu\dusom_glickfeldlab\All_Staff\home\lan\Data\2P_images\mat_inter/'.replace('\\', '/')
    csv_filename = 'df_tidy_continuous_vis_pval.csv'
    try:
        df_tidy_now = pd.read_csv(os.path.join(dir_data, dir_identifier, csv_filename))
    except:
        dir_identifier = dir_identifier + '_cellpose'
        df_tidy_now = pd.read_csv(os.path.join(dir_data, dir_identifier, csv_filename))

    df_tidy_now['mouse'] = mouse
    df_tidy_now['date'] = date
    # df_tidy_now['sess'] = sess
    df_tidy_now['cell_id'] = (df_tidy_now.date.astype(str) + '_' 
                            #    + df_tidy_now.sess.astype(str) + '_' 
                               + df_tidy_now.cell_id.astype(str)) # cell_id adjusted to be unique across mice, dates, sessions
    df_tidy = pd.concat([df_tidy, df_tidy_now], axis=0).reset_index(drop=True)
    # break
    
df_tidy['date_sess'] = df_tidy.date.astype(str) # + '_' + df_tidy.sess.astype(str)
df_tidy

100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


,dfof,cell_id,trial_id,resp_id,isi,stim1_contrast,stim2_id,area,stim1_id,filter_cell_vis,filter_cell_vis_pval,filter_cell_stim,mouse,date,date_sess
0,0.009184,240229_0,1,R1,250,1,7,V1,0,False,0.000136,False,1381,240229,240229
1,-0.011983,240229_0,2,R1,250,1,6,V1,0,False,0.000136,False,1381,240229,240229
2,0.195770,240229_0,3,R1,250,1,3,V1,0,False,0.000136,False,1381,240229,240229
3,0.232615,240229_0,5,R1,250,1,6,V1,0,False,0.000136,False,1381,240229,240229
4,0.061147,240229_0,6,R1,250,1,3,V1,0,False,0.000136,False,1381,240229,240229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213318,0.083449,240229_120,1031,R2,250,1,7,V1,0,False,0.007587,False,1381,240229,240229
213319,0.001555,240229_120,1032,R2,250,1,5,V1,0,False,0.007587,False,1381,240229,240229
213320,0.049180,240229_120,1033,R2,250,1,2,V1,0,False,0.007587,True,1381,240229,240229
213321,-0.077281,240229_120,1034,R2,6000,0,4,V1,0,False,0.007587,False,1381,240229,240229


# build well max

## filter cell well-max (no rerun)

In [41]:
## construct tuning_vec column

max_ori_bootstrap = np.array([])
nboot = 50

for iboot in tqdm(range(nboot)):
    max_ori = (df_tidy[df_tidy.resp_id == 'R2'] # only R2 has diff ori
                [['dfof', 'cell_id', 'resp_id', 'isi', 'stim2_id']]
                .groupby(['cell_id', 'isi', 'stim2_id'])
                .sample(frac=.7, random_state=iboot) # sample x trials per group. min rep = 48
                .groupby(['cell_id', 'isi', 'stim2_id'])
                .agg({'dfof': 'mean'}) # aggregate resp by cell, isi, ori
                .groupby(['cell_id', 'isi']).apply(lambda x: np.argmax(x)) # get ori with max resp
                .reset_index() # NOTE: due to prev groupby, cell_id and isi are in order
                .rename(columns={0: 'max_ori'}).max_ori.values
                )
    max_ori_bootstrap = np.append(max_ori_bootstrap, max_ori)

100%|██████████| 50/50 [00:08<00:00,  5.99it/s]


In [42]:
max_ori_bootstrap = max_ori_bootstrap.reshape(nboot, -1) # reshape max_ori_bootstrap to: nboot x (ncell x nisi)

## sort max_ori_bootstrap in each col (across boots)
max_ori_bootstrap = np.sort(max_ori_bootstrap, axis=0)
# sns.heatmap(max_ori_bootstrap[:, 10:30], cmap='viridis', cbar=True, annot=True);

## count the most freq value in each col (across boots)
max_ori_mode_freq = stats.mode(max_ori_bootstrap, axis=0)[1] # count of mode (freq of most freq max_ori)
filter_cell_well_max = (max_ori_mode_freq >= nboot * 0.7) # NOTE: strictness of well-max can be adjusted here

## merge filter_cell_well_max with df
df_well_max = (df_tidy[df_tidy.resp_id == 'R2'][['cell_id', 'isi']]
                .groupby(['cell_id', 'isi'])
                .first() # get first trial per cell, isi
                .reset_index() # due to prev groupby, cell_id and isi are in the same order as max_ori
                )
df_well_max['filter_cell_well_max'] = filter_cell_well_max
df_well_max

,cell_id,isi,filter_cell_well_max
0,240229_0,250,True
1,240229_0,6000,False
2,240229_1,250,False
3,240229_1,6000,False
4,240229_10,250,True
...,...,...,...
237,240229_97,6000,False
238,240229_98,250,False
239,240229_98,6000,True
240,240229_99,250,True


In [43]:
## inherit df_well_max['filter_cell_well_max']
df_tidy = df_tidy.merge(
      df_well_max, 
      on=['cell_id', 'isi'], how='left') # take filter_cell_well_max from df_well_max, for each cell and isi
df_tidy

,dfof,cell_id,trial_id,resp_id,isi,stim1_contrast,stim2_id,area,stim1_id,filter_cell_vis,filter_cell_vis_pval,filter_cell_stim,mouse,date,date_sess,filter_cell_well_max
0,0.009184,240229_0,1,R1,250,1,7,V1,0,False,0.000136,False,1381,240229,240229,True
1,-0.011983,240229_0,2,R1,250,1,6,V1,0,False,0.000136,False,1381,240229,240229,True
2,0.195770,240229_0,3,R1,250,1,3,V1,0,False,0.000136,False,1381,240229,240229,True
3,0.232615,240229_0,5,R1,250,1,6,V1,0,False,0.000136,False,1381,240229,240229,True
4,0.061147,240229_0,6,R1,250,1,3,V1,0,False,0.000136,False,1381,240229,240229,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213318,0.083449,240229_120,1031,R2,250,1,7,V1,0,False,0.007587,False,1381,240229,240229,True
213319,0.001555,240229_120,1032,R2,250,1,5,V1,0,False,0.007587,False,1381,240229,240229,True
213320,0.049180,240229_120,1033,R2,250,1,2,V1,0,False,0.007587,True,1381,240229,240229,True
213321,-0.077281,240229_120,1034,R2,6000,0,4,V1,0,False,0.007587,False,1381,240229,240229,False


## pref-ori as max-ori (no rerun)
crude preference: take max-resp orientation as the preferred orien

In [24]:
df_tidy

,dfof,cell_id,trial_id,resp_id,isi,stim1_contrast,stim2_id,area,stim1_id,filter_cell_vis,filter_cell_vis_pval,filter_cell_stim,mouse,date,date_sess,filter_cell_well_max,pref_ori_noad,pref_ori_ad_250,pref_ori_ad_750


In [31]:
(df_tidy[(df_tidy.resp_id == 'R2') & (df_tidy.isi == 6000)
         ]
                .groupby(['cell_id', 'stim2_id'])
                [['dfof']].sum().reset_index()
                .groupby('cell_id') # NOTE: stim2_id is sorted due to prev groupby
                ['dfof'].apply(list).to_frame()
                .dfof.apply(lambda x: np.argmax(x)).to_frame()
                .rename(columns={'dfof': 'pref_ori_noad'})
                )

,pref_ori_noad
cell_id,
240229_0,7
240229_1,3
240229_10,2
240229_100,6
240229_101,2
...,...
240229_95,6
240229_96,0
240229_97,4


In [32]:
pref_ori_noad = (df_tidy[(df_tidy.resp_id == 'R2') & (df_tidy.isi == 6000)]
                .groupby(['cell_id', 'stim2_id'])
                [['dfof']].sum().reset_index()
                .groupby('cell_id') # NOTE: stim2_id is sorted due to prev groupby
                ['dfof'].apply(list).to_frame()
                .dfof.apply(lambda x: np.argmax(x)).to_frame()
                .rename(columns={'dfof': 'pref_ori_noad'})
                )

pref_ori_ad_250 = (df_tidy[(df_tidy.resp_id == 'R2') & (df_tidy.isi == 250)]
                .groupby(['cell_id', 'stim2_id'])
                [['dfof']].sum().reset_index()
                .groupby('cell_id') # NOTE: stim2_id is sorted due to prev groupby
                ['dfof'].apply(list).to_frame()
                .dfof.apply(lambda x: np.argmax(x)).to_frame()
                .rename(columns={'dfof': 'pref_ori_ad_250'})
                )

pref_ori_ad_750 = (df_tidy[(df_tidy.resp_id == 'R2') & (df_tidy.isi == 750)]
                .groupby(['cell_id', 'stim2_id'])
                [['dfof']].sum().reset_index()
                .groupby('cell_id') # NOTE: stim2_id is sorted due to prev groupby
                ['dfof'].apply(list).to_frame()
                .dfof.apply(lambda x: np.argmax(x)).to_frame()
                .rename(columns={'dfof': 'pref_ori_ad_750'})
                )

df_tidy = (df_tidy.merge(pref_ori_noad, on='cell_id')
            .merge(pref_ori_ad_250, on='cell_id')
            .merge(pref_ori_ad_750, on='cell_id'))
df_tidy

,dfof,cell_id,trial_id,resp_id,isi,stim1_contrast,stim2_id,area,stim1_id,filter_cell_vis,filter_cell_vis_pval,filter_cell_stim,mouse,date,date_sess,pref_ori_noad,pref_ori_ad_250,pref_ori_ad_750


In [ ]:
def bin_ori(x):
    # bin pref ori to 0, 45, 90
    if x < 30:
        return 0
    elif x <= 60:
        return 45
    else:
        return 90

def distace_from_adapter(x):
    # adapter is always 0 deg
    x = 22.5*x # convert from stim id int to degree
    if x > 90:
        x = 180 - x
    return x


df_tidy['pref_unadapted_distance'] = df_tidy['pref_ori_noad'].apply(lambda x: distace_from_adapter(x)) # unadapted pref ori, distance from adapter
df_tidy['pref_unadapted_distance_bin'] = df_tidy['pref_unadapted_distance'].apply(lambda x: bin_ori(x)) # bin the distance to 0, 45, 90

df_tidy.loc[(df_tidy.isi == 250), 'pref_adapted_distance'] = df_tidy['pref_ori_ad_250'].apply(lambda x: distace_from_adapter(x)) # isi 250 adapted pref ori, distance from adapter # TODO: refactor pref_ori_ad_250 similarly, so it's only one column of pref_ori_ad, but can filter by isi
df_tidy.loc[(df_tidy.isi == 750), 'pref_adapted_distance'] = df_tidy['pref_ori_ad_750'].apply(lambda x: distace_from_adapter(x)) # isi 750 adapted pref ori, distance from adapter

df_tidy['tuning_bias'] = df_tidy['pref_adapted_distance'] - df_tidy['pref_unadapted_distance'] # distance from adapter, adapted - unadapted. if tuning repelled from adapter, this is positive; attracted, negative

df_tidy.sample(5, random_state=0)

,dfof,cell_id,trial_id,resp_id,isi,stim1_contrast,stim2_id,area,stim1_id,filter_cell_vis,...,sess,date_sess,filter_cell_well_max,pref_ori_noad,pref_ori_ad_250,pref_ori_ad_750,pref_unadapted_distance,pref_unadapted_distance_bin,pref_adapted_distance,tuning_bias
1535086,0.070788,230302_004_53,353,R2,750,1,3,LM,0,True,...,004,230302_004,True,5,3,3,67.5,90,67.5,0.0
1498362,-0.001468,230302_004_10,50,R2,250,1,7,LM,0,True,...,004,230302_004,False,0,0,7,0.0,0,0.0,0.0
10251,-0.030888,200720_003_13,84,R1,250,1,6,V1,0,True,...,003,200720_003,True,0,0,0,0.0,0,0.0,0.0
26153,-0.020128,200720_003_33,401,R1,250,1,0,V1,0,False,...,003,200720_003,True,7,7,3,22.5,0,22.5,0.0
1794444,0.021428,230307_004_31,149,R2,250,1,3,LM,0,False,...,004,230307_004,True,6,6,7,45.0,45,45.0,0.0


## read csv for well-max

In [ ]:
dir_data = r'\\duhs-user-nc1.dhe.duke.edu\dusom_glickfeldlab\All_Staff\home\lan\Data\2P_images\mat_inter\df tidy for plots/'.replace('\\', '/')
# df_tidy.to_csv(os.path.join(dir_data, 'df_tidy_tuning_bias_wellmax.csv'), index=False) # takes 80 sec
df_tidy = pd.read_csv(os.path.join(dir_data, 'df_tidy_tuning_bias_wellmax.csv'))
df_tidy.sample(5, random_state=0)

,dfof,cell_id,trial_id,resp_id,isi,stim1_contrast,stim2_id,area,stim1_id,filter_cell_vis,...,date,sess,filter_cell_well_max,pref_ori_noad,pref_ori_ad_250,pref_ori_ad_750,pref_unadapted_distance,pref_unadapted_distance_bin,pref_adapted_distance,tuning_bias
1535086,0.070788,230302_004_53,353,R2,750,1,3,LM,0,True,...,230302,4,True,5,3,3,67.5,90,67.5,0.0
1498362,-0.001468,230302_004_10,50,R2,250,1,7,LM,0,True,...,230302,4,False,0,0,7,0.0,0,0.0,0.0
10251,-0.030888,200720_003_13,84,R1,250,1,6,V1,0,True,...,200720,3,True,0,0,0,0.0,0,0.0,0.0
26153,-0.020128,200720_003_33,401,R1,250,1,0,V1,0,False,...,200720,3,True,7,7,3,22.5,0,22.5,0.0
1794444,0.021428,230307_004_31,149,R2,250,1,3,LM,0,False,...,230307,4,True,6,6,7,45.0,45,45.0,0.0


## write well-max.mat for pop vec decoder

In [47]:
pd.options.mode.chained_assignment = None  # default='warn', suppress SettingWithCopyWarning

df_well_max = df_tidy[df_tidy.isi == 6000] # determine well max cell in ISI=6k
df_well_max['date_sess'] = df_well_max.date.astype(str) # + '_00' + df_well_max.sess.astype(str)

for date_sess in tqdm(df_well_max.date_sess.unique()):

    df_well_max_sess = df_well_max[df_well_max.date_sess == date_sess]
    df_well_max_sess = df_well_max_sess[['date_sess', 'cell_id', 'filter_cell_well_max']]
    df_well_max_sess['cell_id_int'] = df_well_max_sess.cell_id.str.split('_').apply(lambda x: int(x[-1])) # cast cell_id from str to int
    df_well_max_sess = df_well_max_sess.groupby('cell_id_int').first().reset_index() # groupby sorts cell_id
    well_max_sess = df_well_max_sess.filter_cell_well_max.values.astype(float) # well max cell filter in this session. must cast to float for matlab to index with bool

    ## find session subdir in mat_inter
    mat_inter = r'Z:\All_Staff\home\lan\Data\2P_images\mat_inter'.replace('\\', '/')
    for dirname in os.listdir(mat_inter):
        if date_sess in dirname: # this accounts for cellpose vs manual segm
            dir_sess = mat_inter + '/' + dirname
            break
    
    ## save well max .mat
    # sio.savemat(os.path.join(dir_sess, 'well_max.mat'), {'well_max': well_max_sess})

100%|██████████| 1/1 [00:00<00:00, 17.85it/s]
